In [ ]:
# Import Data
image_shape = 300

import numpy as np
X_train = np.load('./mango_train_data.npy',allow_pickle=True)
X_test = np.load('./mango_dev_data.npy',allow_pickle=True)

import cv2
def resize_mango(mango_data, size):
    res_mango = list()
    for mango in mango_data:
        
        
        
        
        large_size = max(int(np.shape(mango)[0]),int(np.shape(mango)[1]))
        mango = cv2.copyMakeBorder(mango,int((large_size-np.shape(mango)[0])/2),int((large_size-np.shape(mango)[0])/2)
                               ,int((large_size-np.shape(mango)[1])/2),int((large_size-np.shape(mango)[1])/2)
                               ,cv2.BORDER_REPLICATE)
        res_mango.append(cv2.resize(mango,(size,size),interpolation = cv2.INTER_CUBIC))    
#     np.save('./res_{}_mango({}).npy'.format(name,size),res_mango)
    return np.array(res_mango)

X_train = resize_mango(X_train,image_shape)
y_train = np.load('./mango_train_label.npy',allow_pickle=True)
X_test = resize_mango(X_test,image_shape)
y_test = np.load('./mango_dev_label.npy',allow_pickle=True)

print (np.shape(X_train))
print (np.shape(y_train))
print (np.shape(X_test))
print (np.shape(y_test))

In [ ]:
# Setting Data Generator
import Augmentor

train = Augmentor.Pipeline()

train.random_brightness(probability=0.6, min_factor=0.9, max_factor=1.2)
train.random_color(probability=0.6, min_factor=0.9, max_factor=1.2)
train.random_contrast(probability=0.6, min_factor=0.95, max_factor=1.1)
train.random_distortion(probability=0.5, grid_width=4, grid_height=8, magnitude=6)
train.skew(probability=0.8, magnitude=0.5)
train.shear(probability=0.7, max_shear_left=20, max_shear_right=20)
train.zoom(probability=0.5, min_factor=1.1, max_factor=1.3)
train.rotate(probability=1, max_left_rotation=24, max_right_rotation=24)
train.rotate_random_90(0.75)
train.zoom(probability=0.2, min_factor=1.1, max_factor=1.3)
train.flip_random(0.75)
train.resize(probability=1, width=image_shape, height=image_shape, resample_filter=u'BICUBIC')

from tensorflow.keras.preprocessing.image import ImageDataGenerator
traingen = ImageDataGenerator(
      preprocessing_function = train.keras_preprocess_func(),
)

test = Augmentor.Pipeline()
test.zoom(probability=1, min_factor=1.1, max_factor=1.2)
test.resize(1.0, image_shape, image_shape)

testgen = ImageDataGenerator(
      preprocessing_function = test.keras_preprocess_func(),  
)

# 從中間切固定大小
# crop_by_size(probability, width, height, centre=True) 
# 從中間擷取隨機大小 percentage_area(隨機的大小)
# crop_centre(probability, percentage_area, randomise_percentage_area=False)
# 隨機擷取隨機大小 
# crop_random(probability, percentage_area, randomise_percentage_area=False)
# 翻轉
# traingen.flip_random(0.75)
# 圖內變形(小區域放大縮小扭曲)
# traingen.gaussian_distortion(0.2, 2, 2, 1, "bell", "in")
# 增加對比度
# histogram_equalisation(probability=1.0)
# 亮度 0黑 1原圖
# random_brightness(probability, min_factor, max_factor)
# 飽和度 1 原圖
# random_color(probability, min_factor, max_factor)
# 對比度 1 原圖
# random_contrast(probability, min_factor, max_factor)
# 圖內形變
# random_distortion(probability=0.5, grid_width=4, grid_height=8, magnitude=5)
# 轉成指定大小
# resize(probability, width, height, resample_filter=u'BICUBIC')
# 隨機旋轉(最多25)
# rotate(probability=1, max_left_rotation=24, max_right_rotation=24)
# 旋轉 90 的倍數
# rotate_random_90(0.75)
# 傾斜
# skew(probability=0.5, magnitude=0.5)
# 放大
# zoom(probability=0.5, min_factor=1.1, max_factor=1.3)


In [ ]:
# Setting New Model
import tensorflow
print(tensorflow.__version__)
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam, SGD
import efficientnet.tfkeras as efn

net = efn.EfficientNetB3( include_top=True, weights='imagenet'
                         , input_tensor=None,input_shape=(image_shape,image_shape,3))
net.summary()

inpt = Input(shape=(image_shape, image_shape, 3))
x = net(inpt)

model = Model(inputs=[inpt], outputs=[x])
model.summary()
model.compile(  loss= ['sparse_categorical_crossentropy'],
                optimizer= Adam(lr=0.0002 ,decay=1e-8),
                metrics=['accuracy']
                )

In [ ]:
# Setting LookBack Function

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.callbacks import Callback
import os

LR_function=ReduceLROnPlateau(
    monitor='val_loss',                        
    patience= 4, # 3 epochs 內acc沒下降就要調整LR
    verbose=1,    
    factor=0.5, # LR降為0.5
#     min_lr=0.00000001
)

earlystop_function = EarlyStopping(
    monitor='val_loss', 
#     min_delta= 0,  #容忍漲跌幅範圍
    patience= 10,  #容忍不下降回合數
    verbose= 1, 
    mode='min',
#     baseline=None 
    restore_best_weights = True
)

class save_best_model(Callback):
    def on_epoch_end(self, epoch, logs=None):
        global best_loss
        global best_acc
        
        if best_loss > logs['val_loss']:  
            filepath='./Trained_model/{}Best_loss[{:.4f}].h5'.format(saving_name, best_loss)
            try:
                os.remove(filepath)
            except:
                None
            best_loss = logs['val_loss']
            filepath='./Trained_model/{}Best_loss[{:.4f}].h5'.format(saving_name, best_loss)
            model.save(filepath)
            print ('\nSave',filepath)
        else:
            print ('\nNot better loss than {:.4f}'.format(best_loss))
            
        if best_acc < logs['val_accuracy']:
            filepath='./Trained_model/{}Best_acc[{:.4f}].h5'.format(saving_name, best_acc*100)
            try:
                os.remove(filepath)
            except:
                None
            best_acc = logs['val_accuracy']
            filepath='./Trained_model/{}Best_acc[{:.4f}].h5'.format(saving_name, best_acc*100)
            model.save(filepath)
            print ('Save',filepath)
        else:
            print ('Not better acc than {:.4f}'.format(best_acc))
            

In [ ]:
# Change Learning rate during Trainging

import keras.backend as K
K.set_value(model.optimizer.lr, 0.0002500000118743628 )

print (K.get_value(model.optimizer.lr))

In [ ]:
# Training Model
for i in range(6,10):
    saving_name = 'Mango_EfficientNetB2[{}]'.format(i)
################

    import tensorflow as tf
    best_loss = 100.
    best_acc = 0.
    tf.keras.backend.clear_session()
    net = efn.EfficientNetB2( include_top=True, weights='imagenet'
                         , input_tensor=None,input_shape=(image_shape,image_shape,3))
    inpt = Input(shape=(image_shape, image_shape, 3))
    x = net(inpt)
    model = Model(inputs=[inpt], outputs=[x])
    model.compile(  loss= ['sparse_categorical_crossentropy'],
                optimizer= Adam(lr=0.0002 ,decay=1e-8),
                metrics=['accuracy']
                )

    model.fit(traingen.flow(X_train,y_train,batch_size=35), 
                    steps_per_epoch=X_train.shape[0]/35 , epochs=300, shuffle = True,
                    validation_data=testgen.flow(X_test,y_test,batch_size=32,shuffle=False),
                    validation_steps=X_test.shape[0]/32,
                    callbacks=[save_best_model(), LR_function, 
                               earlystop_function],
                    workers = 10)

In [ ]:
# Evaluate Model Score
batch_size = 32

#########################
score = model.evaluate(testgen.flow(X_train,y_train,batch_size=batch_size,shuffle=False)
                      ,steps = X_train.shape[0]/batch_size)
# print (score)
# score = model.evaluate(X_train/255.,y_train)
# print (score)
score = model.evaluate(testgen.flow(X_test,y_test,batch_size=batch_size,shuffle=False)
                       ,steps = X_test.shape[0]/batch_size)
# print (score)
# score = model.evaluate(X_test/255.,y_test)
# print (score)

In [ ]:
# Saving Model
model_name = 'adi_B3[5]Best_loss[0.3885].h5'
saving_path = './Model/'+ model_name +'.h5'

#################################
model.save(saving_path)

In [ ]:
# Import Test Data

test_data = np.load('./mango_test_data.npy',allow_pickle=True)
test_name = np.load('./mango_test_name.npy',allow_pickle=True)
test_data = resize_mango(test_data,image_shape)

In [ ]:
# Get Test Result

ans = model.predict(testgen.flow(test_data,batch_size=batch_size,shuffle=False))
# ans = model.predict(test_data/255.)

np.save('./Predict/' + model_name + 'predict.npy',ans[:,1:4])

ans = np.argmax(ans, axis=1)
ans = list(ans)

output = list()
output.append (['image_id', 'label'])
for idx,data in enumerate(ans):
    if ans[idx] == 1:
        label = 'A'
    if ans[idx] == 2:
        label = 'B'
    if ans[idx] == 3:
        label = 'C'
    output.append([test_name[idx],label])

import csv

with open( './Result/' + model_name + 'ans.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerows(output)

In [ ]:
# Load Model

saving_name = ''

from tensorflow.keras.models import load_model
model = load_model('./Trained_model/Mango_EfficientNetB3[5]Best_loss[0.3885].h5')